In [26]:
# pip install requests
import requests
# pip install parsel 
from parsel import Selector

In [27]:
faqEntries = {}
def extractEntries(currentPage):
    currentEntries = currentPage.css('span a[href]')
    for e in currentEntries:
        link = e.xpath('@href').get()
        question = e.xpath('text()').get()
        faqEntries[link] = question


Load initial page

In [28]:
response = requests.get('https://www.faq.bag.admin.ch/en/categories/vaccination')
selector = Selector(response.text)

Load questions of initial page

In [29]:
extractEntries(selector)

Load other available pages

In [32]:
faqPageSelectors = selector.css('a[title][href]')
faqPageUrls = []
for p in faqPageSelectors:
    url = p.xpath('@href').get()
    if "page" in url:
        response = requests.get('https://www.faq.bag.admin.ch'+url)
        selector = Selector(response.text)
        extractEntries(selector)

Count available questions

In [34]:
len(faqEntries)

60

Extract answers

In [51]:
class FAQEntry:
    link = ''
    question = ''
    answer = ''

finalEntries = []
for e in faqEntries:
    link = 'https://www.faq.bag.admin.ch'+e;
    response = requests.get(link)
    selector = Selector(response.text)
    answer = selector.css('.field-item.even p').xpath('text()').get()
    if answer:
        entry = FAQEntry()
        entry.link = link
        entry.question = faqEntries[e]
        entry.answer = answer
        finalEntries.append(entry)

Print extracted entries

In [52]:
for e in finalEntries:
    print("Link: "+ e.link)
    print("Question: "+ e.question)
    print("Answer: "+ e.answer)
    print("----------------")

Link: https://www.faq.bag.admin.ch/en/vaccination-registration-and-organisation/how-and-where-can-i-register-vaccination
Question: How and where can I register for the vaccination?
Answer: You will find information on how vaccinations are organised in your canton and where you can register on the website of your canton. You’ll find a list of cantonal websites here: 
----------------
Link: https://www.faq.bag.admin.ch/en/vaccination-change-measures-vaccinated-people/do-i-still-have-go-quarantineisolation-if-i-have-been
Question: Do I still have to go into quarantine/isolation if I have been vaccinated?
Answer: The 
----------------
Link: https://www.faq.bag.admin.ch/en/vaccination-vaccination-recommendations-and-strategy/which-persons-are-not-being-vaccinated-present
Question: Which persons are not being vaccinated at present?
Answer: The following persons should not be vaccinated against COVID-19 with the approved Pfizer/BioNTech and Moderna vaccines:
----------------
Link: https://www